# Week 2 Lab: Data Validation & Quality

**CS 203: Software Tools and Techniques for AI**

---

## Lab Overview

In this lab, you will learn to validate and clean data using:
1. **Unix CLI tools** - Quick data inspection
2. **jq** - JSON processing
3. **pandas** - Data profiling and cleaning
4. **Pydantic** - Schema validation
5. **Great Expectations** - Data quality testing

**Goal**: Transform messy movie data into clean, validated data ready for ML.

---

## Setup

In [ ]:
# Install required packages (run this first!)
!sudo apt-get install -y jq -qq  # JSON processor for CLI
!pip install -q pandas pydantic csvkit

# Verify installations
!echo "jq version:" && jq --version
!echo "csvkit version:" && csvlook --version 2>&1 | head -1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.7/198.7 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.6/114.6 kB 8.9 MB/s eta 0:00:00
jq version:
jq-1.6
csvkit version:
csvlook 2.2.0


In [ ]:
import pandas as pd
import numpy as np
import json
from typing import Optional, List
from pydantic import BaseModel, Field, validator, ValidationError

print("All imports successful!")

All imports successful!


---

# Part 1: Sample Messy Data

Let's create some realistic messy movie data that mimics what you'd get from APIs.

In [ ]:
# Create sample messy data (simulating what we collected in Week 1)
messy_movies = [
    {"Title": "Inception", "Year": "2010", "Runtime": "148 min", "imdbRating": "8.8", "BoxOffice": "$292,576,195", "Genre": "Action, Sci-Fi"},
    {"Title": "Avatar", "Year": "2009", "Runtime": "162 min", "imdbRating": "7.9", "BoxOffice": "$760,507,625", "Genre": "Action, Adventure"},
    {"Title": "The Room", "Year": "2003", "Runtime": "99 min", "imdbRating": "3.9", "BoxOffice": "N/A", "Genre": "Drama"},
    {"Title": "Inception", "Year": "2010", "Runtime": "148 min", "imdbRating": "8.8", "BoxOffice": "$292,576,195", "Genre": "Action, Sci-Fi"},  # Duplicate!
    {"Title": "Tenet", "Year": "N/A", "Runtime": "150 min", "imdbRating": "7.3", "BoxOffice": "N/A", "Genre": "Action, Sci-Fi"},  # Missing year
    {"Title": "The Matrix", "Year": "1999", "Runtime": "136 min", "imdbRating": "8.7", "BoxOffice": "$171,479,930", "Genre": "Action, Sci-Fi"},
    {"Title": "Interstellar", "Year": "2014", "Runtime": "", "imdbRating": "8.6", "BoxOffice": "$188,020,017", "Genre": "Adventure, Drama"},  # Empty runtime
    {"Title": "", "Year": "2020", "Runtime": "120 min", "imdbRating": "7.0", "BoxOffice": "$50,000,000", "Genre": "Comedy"},  # Missing title!
    {"Title": "Joker", "Year": "2019", "Runtime": "122 min", "imdbRating": "invalid", "BoxOffice": "$335,451,311", "Genre": "Crime, Drama"},  # Invalid rating
    {"Title": "Parasite", "Year": "2019", "Runtime": "132 min", "imdbRating": "8.5", "BoxOffice": "$53,369,749", "Genre": "Drama, Thriller"},
]

# Save as JSON for CLI exercises
with open('messy_movies.json', 'w') as f:
    json.dump(messy_movies, f, indent=2)

# Create DataFrame
df_messy = pd.DataFrame(messy_movies)
df_messy.to_csv('messy_movies.csv', index=False)

print("Created messy_movies.json and messy_movies.csv")
print(f"\nDataFrame shape: {df_messy.shape}")
df_messy

Created messy_movies.json and messy_movies.csv

DataFrame shape: (10, 6)


,Title,Year,Runtime,imdbRating,BoxOffice,Genre
0,Inception,2010,148 min,8.8,"$292,576,195","Action, Sci-Fi"
1,Avatar,2009,162 min,7.9,"$760,507,625","Action, Adventure"
2,The Room,2003,99 min,3.9,N/A,Drama
3,Inception,2010,148 min,8.8,"$292,576,195","Action, Sci-Fi"
4,Tenet,N/A,150 min,7.3,N/A,"Action, Sci-Fi"
5,The Matrix,1999,136 min,8.7,"$171,479,930","Action, Sci-Fi"
6,Interstellar,2014,,8.6,"$188,020,017","Adventure, Drama"
7,,2020,120 min,7.0,"$50,000,000",Comedy
8,Joker,2019,122 min,invalid,"$335,451,311","Crime, Drama"
9,Parasite,2019,132 min,8.5,"$53,369,749","Drama, Thriller"


---

# Part 2: CLI Data Inspection

Before writing code, always inspect your data with CLI tools.

### Question 2.1 (Solved): Basic File Inspection

In [ ]:
# SOLVED EXAMPLE
# Check file sizes and line counts
!echo "=== File sizes ==="
!ls -lh messy_movies.json messy_movies.csv

!echo "\n=== Line counts ==="
!wc -l messy_movies.json messy_movies.csv

!echo "\n=== First 5 lines of CSV ==="
!head -5 messy_movies.csv

=== File sizes ===
-rw-r--r-- 1 root root  572 Jan 29 16:15 messy_movies.csv
-rw-r--r-- 1 root root 1.6K Jan 29 16:15 messy_movies.json
\n=== Line counts ===
  81 messy_movies.json
  11 messy_movies.csv
  92 total
\n=== First 5 lines of CSV ===
Title,Year,Runtime,imdbRating,BoxOffice,Genre
Inception,2010,148 min,8.8,"$292,576,195","Action, Sci-Fi"
Avatar,2009,162 min,7.9,"$760,507,625","Action, Adventure"
The Room,2003,99 min,3.9,N/A,Drama
Inception,2010,148 min,8.8,"$292,576,195","Action, Sci-Fi"


In [ ]:
!csvstat messy_movies.csv | grep "Type of data"

	Type of data:          Text
	Type of data:          Number
	Type of data:          TimeDelta
	Type of data:          Text
	Type of data:          Number
	Type of data:          Text


### Question 2.2: Inspect JSON with jq

Use `jq` to:
1. Pretty-print the JSON file
2. Get the length (number of movies)
3. Extract just the titles

In [ ]:
# YOUR CODE HERE
# Pretty print
print("=============Pretty print=============\n")
! cat messy_movies.json | jq .
# Get length
print("\n=============Number of Movies=============\n")
! cat messy_movies.json | jq 'length'

# Extract titles
print("\n =============Titles============= \n")
! cat messy_movies.json | jq '.[].Title'


=============Pretty print=============

[
  {
    "Title": "Inception",
    "Year": "2010",
    "Runtime": "148 min",
    "imdbRating": "8.8",
    "BoxOffice": "$292,576,195",
    "Genre": "Action, Sci-Fi"
  },
  {
    "Title": "Avatar",
    "Year": "2009",
    "Runtime": "162 min",
    "imdbRating": "7.9",
    "BoxOffice": "$760,507,625",
    "Genre": "Action, Adventure"
  },
  {
    "Title": "The Room",
    "Year": "2003",
    "Runtime": "99 min",
    "imdbRating": "3.9",
    "BoxOffice": "N/A",
    "Genre": "Drama"
  },
  {
    "Title": "Inception",
    "Year": "2010",
    "Runtime": "148 min",
    "imdbRating": "8.8",
    "BoxOffice": "$292,576,195",
    "Genre": "Action, Sci-Fi"
  },
  {
    "Title": "Tenet",
    "Year": "N/A",
    "Runtime": "150 min",
    "imdbRating": "7.3",
    "BoxOffice": "N/A",
    "Genre": "Action, Sci-Fi"
  },
  {
    "Title": "The Matrix",
    "Year": "1999",
    "Runtime": "136 min",
    "imdbRating": "8.7",
    "BoxOffice": "$171,479,930",
    "Genre":

### Question 2.3: Find Data Issues with jq

Use `jq` to find:
1. Movies where Year is "N/A"
2. Movies where BoxOffice is "N/A"
3. Movies where Title is empty

**Hint**: Use `select()` function in jq

In [ ]:
# YOUR CODE HERE
print('\n==========Movies where Year is "N/A"==========\n')
!cat messy_movies.json | jq '.[]  | select(.Year == "N/A")'
print('\n==========Movies where BoxOffice is "N/A"==========\n')
!cat messy_movies.json | jq '.[] | select(.BoxOffice == "N/A")'
print('\n==========Movies where Title is "N/A"==========\n')
!cat messy_movies.json | jq '.[] | select(.Title == "")'


==========Movies where Year is "N/A"==========

{
  "Title": "Tenet",
  "Year": "N/A",
  "Runtime": "150 min",
  "imdbRating": "7.3",
  "BoxOffice": "N/A",
  "Genre": "Action, Sci-Fi"
}

==========Movies where BoxOffice is "N/A"==========

{
  "Title": "The Room",
  "Year": "2003",
  "Runtime": "99 min",
  "imdbRating": "3.9",
  "BoxOffice": "N/A",
  "Genre": "Drama"
}
{
  "Title": "Tenet",
  "Year": "N/A",
  "Runtime": "150 min",
  "imdbRating": "7.3",
  "BoxOffice": "N/A",
  "Genre": "Action, Sci-Fi"
}

==========Movies where Title is "N/A"==========

{
  "Title": "",
  "Year": "2020",
  "Runtime": "120 min",
  "imdbRating": "7.0",
  "BoxOffice": "$50,000,000",
  "Genre": "Comedy"
}


---

# Part 3: Data Profiling with Pandas

Now let's do systematic data profiling.

### Question 3.1 (Solved): Basic Data Profiling

In [ ]:
# SOLVED EXAMPLE
print("=== Data Types ===")
print(df_messy.dtypes)

print("\n=== Missing Values ===")
print(df_messy.isnull().sum())

print("\n=== Unique Values per Column ===")
print(df_messy.nunique())

print("\n=== Sample Values ===")
for col in df_messy.columns:
    print(f"{col}: {df_messy[col].unique()[:5]}")

=== Data Types ===
Title         object
Year          object
Runtime       object
imdbRating    object
BoxOffice     object
Genre         object
dtype: object

=== Missing Values ===
Title         0
Year          0
Runtime       0
imdbRating    0
BoxOffice     0
Genre         0
dtype: int64

=== Unique Values per Column ===
Title         9
Year          8
Runtime       9
imdbRating    9
BoxOffice     8
Genre         7
dtype: int64

=== Sample Values ===
Title: ['Inception' 'Avatar' 'The Room' 'Tenet' 'The Matrix']
Year: ['2010' '2009' '2003' 'N/A' '1999']
Runtime: ['148 min' '162 min' '99 min' '150 min' '136 min']
imdbRating: ['8.8' '7.9' '3.9' '7.3' '8.7']
BoxOffice: ['$292,576,195' '$760,507,625' 'N/A' '$171,479,930' '$188,020,017']
Genre: ['Action, Sci-Fi' 'Action, Adventure' 'Drama' 'Adventure, Drama' 'Comedy']


### Question 3.2: Identify All Data Quality Issues

Write a function `profile_data(df)` that returns a dictionary summarizing:
1. Total rows
2. Duplicate rows
3. Missing values per column (including "N/A" strings)
4. Empty strings per column
5. Data type issues (strings that should be numbers)

In [ ]:
# YOUR CODE HERE
def profile_data(df):
    """Generate a data quality profile."""
    profile = {}

    profile["total_rows"] = df.shape[0]
    profile["duplicate_rows"] = len(df[df.duplicated()].index.tolist())
    profile["missing_values"] = {
        col: int(((df[col].isna()) | (df[col] == 'N/A') | (df[col] == "invalid")).sum())
        for col in df.columns
    }
    profile["empty_strings"] = {
        col: int(((df[col] == "")).sum())
        for col in df.columns
    }

    data_type_issues = {}
    for col in df.columns:
      if df[col].dtype == object:
        series = df[col]
        series = series[~((series.isna()) | (series == "") | (series == "N/A"))]
        series = pd.to_numeric(series, errors = 'coerce')
        issues = int(series.isna().sum())
        if issues > 0 and issues < series.shape[0]:
          data_type_issues[col] = issues

    profile["data_type_issues"] = data_type_issues

    return profile

# Test
profile = profile_data(df_messy)
profile
print(json.dumps(profile, indent=2))

{
  "total_rows": 10,
  "duplicate_rows": 1,
  "missing_values": {
    "Title": 0,
    "Year": 1,
    "Runtime": 0,
    "imdbRating": 1,
    "BoxOffice": 2,
    "Genre": 0
  },
  "empty_strings": {
    "Title": 1,
    "Year": 0,
    "Runtime": 1,
    "imdbRating": 0,
    "BoxOffice": 0,
    "Genre": 0
  },
  "data_type_issues": {
    "imdbRating": 1
  }
}


### Question 3.3: Find Duplicates

Find all duplicate rows in the dataset. How many duplicates are there? Which movies are duplicated?

In [ ]:
# YOUR CODE HERE
duplicated_rows = df_messy[df_messy.duplicated()]
print("\n==========Duplicated Row==========\n")
print(duplicated_rows)
print("\n==========Total Duplicates==========\n")
print(duplicated_rows.shape[0])
print("\n==========Duplicated Movies title==========\n")
print(duplicated_rows['Title'])


==========Duplicated Row==========

       Title  Year  Runtime imdbRating     BoxOffice           Genre
3  Inception  2010  148 min        8.8  $292,576,195  Action, Sci-Fi

==========Total Duplicates==========

1

==========Duplicated Movies title==========

3    Inception
Name: Title, dtype: object


---

# Part 4: Data Cleaning

Now let's clean the data systematically.

### Question 4.1 (Solved): Clean Runtime Column

In [ ]:
# SOLVED EXAMPLE
def clean_runtime(runtime_str):
    """Convert '148 min' to integer 148."""
    if pd.isna(runtime_str) or runtime_str == '' or runtime_str == 'N/A':
        return None
    # Extract digits
    import re
    match = re.search(r'(\d+)', str(runtime_str))
    if match:
        return int(match.group(1))
    return None

# Test
print(clean_runtime('148 min'))  # 148
print(clean_runtime('N/A'))      # None
print(clean_runtime(''))         # None

148
None
None


### Question 4.2: Clean BoxOffice Column

Write a function `clean_box_office(value)` that converts:
- `"$292,576,195"` → `292576195` (integer)
- `"N/A"` → `None`
- `""` → `None`

In [ ]:
# YOUR CODE HERE
def clean_box_office(value):
    """Convert '$292,576,195' to integer."""
    if pd.isna(value) or value == '' or value == 'N/A':
        return None

    return int(value.replace(',', '').replace('$', ''))


# Test
print(clean_box_office('$292,576,195'))  # 292576195
print(clean_box_office('N/A'))           # None

292576195
None


### Question 4.3: Clean Year Column

Write a function `clean_year(value)` that:
- Converts valid year strings to integers
- Returns `None` for "N/A" or invalid values
- Validates that year is between 1888 (first film) and current year + 2

In [ ]:
from datetime import date
# YOUR CODE HERE
def clean_year(value):
    """Convert '2007' to integer."""
    if pd.isna(value) or value == '' or value == 'N/A':
        return None

    year = int(value)
    if year < 1888 or year > date.today().year + 2:
      return None

    return year

print(clean_year('1987'))
print(clean_year('1887'))
print(clean_year('N/A'))
print(clean_year('2030'))

1987
None
None
None


### Question 4.4: Clean Rating Column

Write a function `clean_rating(value)` that:
- Converts valid rating strings to floats
- Returns `None` for invalid values
- Validates that rating is between 0.0 and 10.0

In [ ]:
# YOUR CODE HERE
def clean_rating(value):
    """Convert '8.8' to integer."""
    if pd.isna(value) or value == '' or value == 'N/A' or value == 'invalid':
        return None

    rating = float(value)
    if rating < 0.0 or rating > 10.0:
      return None

    return rating

print(clean_rating('8.8'))
print(clean_rating(8.8))
print(clean_rating('N/A'))
print(clean_rating('10.5'))

8.8
8.8
None
None


### Question 4.5: Complete Cleaning Pipeline

Create a function `clean_movie_data(df)` that:
1. Removes duplicates
2. Removes rows with empty titles
3. Cleans all columns using the functions above
4. Returns a clean DataFrame with proper data types

In [ ]:
# YOUR CODE HERE
def clean_movie_data(df):
    """Complete cleaning pipeline."""
    df = df.drop_duplicates(keep='first').reset_index(drop=True)
    df = df[df['Title'] != ""].reset_index(drop=True)
    df['Runtime'] = df['Runtime'].apply(lambda x: clean_runtime(x))
    df['imdbRating'] = df['imdbRating'].apply(lambda x: clean_rating(x))
    df['BoxOffice'] = df['BoxOffice'].apply(lambda x: clean_box_office(x))
    df['Year'] = df['Year'].apply(lambda x: clean_year(x))

    return df

# Test
df_clean = clean_movie_data(df_messy)
print(f"Before: {len(df_messy)} rows")
print(f"After: {len(df_clean)} rows")
df_clean

Before: 10 rows
After: 8 rows


,Title,Year,Runtime,imdbRating,BoxOffice,Genre
0,Inception,2010.0,148.0,8.8,292576195.0,"Action, Sci-Fi"
1,Avatar,2009.0,162.0,7.9,760507625.0,"Action, Adventure"
2,The Room,2003.0,99.0,3.9,NaN,Drama
3,Tenet,NaN,150.0,7.3,NaN,"Action, Sci-Fi"
4,The Matrix,1999.0,136.0,8.7,171479930.0,"Action, Sci-Fi"
5,Interstellar,2014.0,NaN,8.6,188020017.0,"Adventure, Drama"
6,Joker,2019.0,122.0,NaN,335451311.0,"Crime, Drama"
7,Parasite,2019.0,132.0,8.5,53369749.0,"Drama, Thriller"


---

# Part 5: Schema Validation with Pydantic

Pydantic provides type validation and data parsing.

### Question 5.1 (Solved): Define a Movie Schema

In [ ]:
# SOLVED EXAMPLE
from pydantic import BaseModel, Field, field_validator
from typing import Optional
import re

class Movie(BaseModel):
    """Validated movie schema."""
    title: str = Field(..., min_length=1, description="Movie title")
    year: int = Field(..., ge=1888, le=2030, description="Release year")
    runtime_minutes: Optional[int] = Field(None, ge=1, le=1000)
    imdb_rating: Optional[float] = Field(None, ge=0, le=10)
    box_office: Optional[int] = Field(None, ge=0)
    genre: str = Field(..., min_length=1)

    @field_validator('title')
    @classmethod
    def title_not_empty(cls, v):
        if not v or not v.strip():
            raise ValueError('Title cannot be empty')
        return v.strip()

# Test with valid data
movie = Movie(
    title="Inception",
    year=2010,
    runtime_minutes=148,
    imdb_rating=8.8,
    box_office=292576195,
    genre="Action, Sci-Fi"
)
print("Valid movie:")
print(movie.model_dump())

Valid movie:
{'title': 'Inception', 'year': 2010, 'runtime_minutes': 148, 'imdb_rating': 8.8, 'box_office': 292576195, 'genre': 'Action, Sci-Fi'}


### Question 5.2: Test Validation Errors

Try creating Movie objects with invalid data and observe the validation errors:
1. Empty title
2. Year before 1888
3. Rating above 10
4. Negative box office

In [ ]:
print("\n==========Empty Title==========\n")
try:
  movie = Movie(
      title="",
      year=2010,
      runtime_minutes=148,
      imdb_rating=8.8,
      box_office=292576195,
      genre="Action, Sci-Fi"
  )
except Exception as e:
  print(e)

print("\n==========Year Before 1888==========\n")
try:
  movie = Movie(
      title="Inception",
      year=1700,
      runtime_minutes=148,
      imdb_rating=8.8,
      box_office=292576195,
      genre="Action, Sci-Fi"
  )
except Exception as e:
  print(e)

print("\n==========Rating above 10==========\n")
try:
  movie = Movie(
      title="Inception",
      year=2010,
      runtime_minutes=148,
      imdb_rating=11.8,
      box_office=292576195,
      genre="Action, Sci-Fi"
  )
except Exception as e:
  print(e)

print("\n==========Negative Box Office==========\n")
try:
  movie = Movie(
      title="Inception",
      year=2010,
      runtime_minutes=148,
      imdb_rating=8.8,
      box_office=-292576195,
      genre="Action, Sci-Fi"
  )
except Exception as e:
  print(e)


==========Empty Title==========

1 validation error for Movie
title
  String should have at least 1 character [type=string_too_short, input_value='', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/string_too_short

==========Year Before 1888==========

1 validation error for Movie
year
  Input should be greater than or equal to 1888 [type=greater_than_equal, input_value=1700, input_type=int]
    For further information visit https://errors.pydantic.dev/2.12/v/greater_than_equal

==========Rating above 10==========

1 validation error for Movie
imdb_rating
  Input should be less than or equal to 10 [type=less_than_equal, input_value=11.8, input_type=float]
    For further information visit https://errors.pydantic.dev/2.12/v/less_than_equal

==========Negative Box Office==========

1 validation error for Movie
box_office
  Input should be greater than or equal to 0 [type=greater_than_equal, input_value=-292576195, input_type=int]
    For further inf

### Question 5.3: Validate and Convert Raw Data

Write a function `validate_movies(raw_data)` that:
1. Takes a list of raw movie dictionaries (from JSON)
2. Attempts to clean and validate each movie
3. Returns two lists: `valid_movies` and `invalid_movies` (with error messages)

In [ ]:
# YOUR CODE HERE
def validate_movies(raw_data):
    """Validate a list of raw movie dictionaries."""
    valid_movies = []
    invalid_movies = []

    # Data Validation
    for i, movie in raw_data.iterrows():
      try:
        movieobj = Movie(title=movie['Title'], year=movie['Year'], runtime_minutes=movie['Runtime'], imdb_rating=movie['imdbRating'], box_office=movie['BoxOffice'], genre=movie['Genre'])
        valid_movies.append(movie)
      except:
        invalid_movies.append(movie)


    return valid_movies, invalid_movies

# Test
# Data Cleaning
raw_data = clean_movie_data(pd.DataFrame(messy_movies))
raw_data['Runtime']
valid, invalid = validate_movies(raw_data)
print(f"Valid: {len(valid)}, Invalid: {len(invalid)}")

Valid: 4, Invalid: 4


---

# Part 6: Data Quality Assertions

Write assertions that should pass for clean data.

### Question 6.1 (Solved): Data Quality Checks

In [ ]:
# SOLVED EXAMPLE
def check_data_quality(df):
    """Run data quality assertions."""
    checks = []

    # Check 1: No duplicate rows
    duplicates = df.duplicated().sum()
    checks.append(("No duplicates", duplicates == 0, f"Found {duplicates} duplicates"))

    # Check 2: No empty titles
    empty_titles = (df['Title'] == '').sum() + df['Title'].isna().sum()
    checks.append(("No empty titles", empty_titles == 0, f"Found {empty_titles} empty titles"))

    # Check 3: Year in valid range
    if 'Year' in df.columns:
        invalid_years = ((df['Year'] < 1888) | (df['Year'] > 2030)).sum()
        checks.append(("Valid years", invalid_years == 0, f"Found {invalid_years} invalid years"))

    # Print results
    print("Data Quality Checks:")
    print("-" * 50)
    for name, passed, message in checks:
        status = "✓" if passed else "✗"
        print(f"{status} {name}: {message if not passed else 'OK'}")

    return all(passed for _, passed, _ in checks)

# Test on messy data (should fail)
print("Checking messy data:")
check_data_quality(df_messy)

### Question 6.2: Add More Quality Checks

Extend the `check_data_quality` function to include:
1. Rating values between 0 and 10
2. No negative box office values
3. Runtime between 1 and 1000 minutes
4. At least 90% of rows have non-null ratings

In [ ]:
# YOUR CODE HERE
# SOLVED EXAMPLE
def check_data_quality(df):
    """Run data quality assertions."""
    checks = []

    # Check 1: No duplicate rows
    duplicates = df.duplicated().sum()
    checks.append(("No duplicates", duplicates == 0, f"Found {duplicates} duplicates"))

    # Check 2: No empty titles
    empty_titles = (df['Title'] == '').sum() + df['Title'].isna().sum()
    checks.append(("No empty titles", empty_titles == 0, f"Found {empty_titles} empty titles"))

    # # Check 3: Year in valid range
    if 'Year' in df.columns:
        years = pd.to_numeric(df['Year'], errors='coerce')
        invalid_years = years.isna().sum() + ((years < 1888) | (years > 2030)).sum()
        checks.append(("Valid years", invalid_years == 0, f"Found {invalid_years} invalid years"))

    # # Check 4: Rating in valid range
    if 'imdbRating' in df.columns:
        ratings = pd.to_numeric(df['imdbRating'], errors='coerce')
        invalid_ratings = ratings.isna().sum() + ((ratings < 0.0) | (ratings > 10.0)).sum()
        checks.append(("Valid ratings", invalid_ratings == 0, f"Found {invalid_ratings} invalid ratings"))

    # # Check 5: BoxOffice in valid range
    if 'BoxOffice' in df.columns:
        boxoffice = pd.to_numeric(df['BoxOffice'], errors='coerce')
        invalid_boxoffice = boxoffice.isna().sum() + ((boxoffice < 0)).sum()
        checks.append(("Valid boxoffice", invalid_boxoffice == 0, f"Found {invalid_boxoffice} invalid BoxOffice"))

    # # Check 6: Runtime in valid range
    if 'Runtime' in df.columns:
        runtime = pd.to_numeric(df['Runtime'], errors='coerce')
        invalid_runtime = runtime.isna().sum() + ((runtime < 1) | (runtime > 1000)).sum()
        checks.append(("Valid runtime", invalid_runtime == 0, f"Found {invalid_runtime} invalid runtime"))

    # # Check 6: Null Rows check
    null_check = df.notnull().values.mean()
    checks.append(("Valid Nulls", null_check > 0.90, f"Found {100 - null_check*100}% null rows"))

    # Print results
    print("Data Quality Checks:")
    print("-" * 50)
    for name, passed, message in checks:
        status = "✓" if passed else "✗"
        print(f"{status} {name}: {message if not passed else 'OK'}")

    return all(passed for _, passed, _ in checks)

# Test on messy data (should fail)
print("Checking messy data:")
check_data_quality(df_clean)

Checking messy data:
Data Quality Checks:
--------------------------------------------------
✓ No duplicates: OK
✓ No empty titles: OK
✗ Valid years: Found 1 invalid years
✗ Valid ratings: Found 1 invalid ratings
✗ Valid boxoffice: Found 2 invalid BoxOffice
✗ Valid runtime: Found 1 invalid runtime
✗ Valid Nulls: Found 10.416666666666657% null rows


False

---

# Part 7: Complete Pipeline

Put everything together into a complete validation pipeline.

### Question 7.1: Build the Complete Pipeline

Create a `DataValidationPipeline` class that:
1. Loads data from JSON or CSV
2. Profiles the data
3. Cleans the data
4. Validates with Pydantic
5. Runs quality checks
6. Exports clean data

In [ ]:
# YOUR CODE HERE
class DataValidationPipeline:
    """Complete data validation pipeline."""

    def __init__(self, input_file):
        self.input_file = input_file
        self.raw_data = None
        self.clean_data = None

    def load(self):
        """Load data from file."""
        if self.input_file.split(".")[-1] == 'json':
          self.raw_data = pd.read_json(self.input_file)

        elif self.input_file.split(".")[-1] == 'csv':
          self.raw_data = pd.read_csv(self.input_file)

        else:
          print("No Valid File Path")

    def profile(self):
        """Generate data profile."""
        print(profile_data(self.raw_data))

    def clean(self):
        """Clean the data."""
        self.clean_data = clean_movie_data(self.raw_data)

    def validate(self):
        """Validate with Pydantic."""
        valid, invalid = validate_movies(self.clean_data)
        print("Valid Movies:", len(valid))
        print("Invalid Movies:", len(invalid))

    def check_quality(self):
        """Run quality checks."""
        check_data_quality(self.clean_data)

    def export(self, output_file):
        """Export clean data."""
        if output_file.split(".")[-1] == 'json':
          self.clean_data.to_json(output_file, orient='records', indent=4)

        elif output_file.split(".")[-1] == 'csv':
          self.clean_data.to_csv(output_file, index=False)

        else:
          print("No Valid File Path")

    def run(self, output_file):
        """Run complete pipeline."""
        self.load()
        print("\n1. Data Profile:")
        self.profile()
        print("\n2. Cleaning...")
        self.clean()
        print("\n3. Validation...")
        self.validate()
        print("\n4. Quality Checks:")
        self.check_quality()
        print("\n5. Exporting...")
        self.export(output_file)

# Test
pipeline = DataValidationPipeline('messy_movies.json')
pipeline.run('clean_movies.csv')


1. Data Profile:
{'total_rows': 10, 'duplicate_rows': 1, 'missing_values': {'Title': 0, 'Year': 1, 'Runtime': 0, 'imdbRating': 1, 'BoxOffice': 2, 'Genre': 0}, 'empty_strings': {'Title': 1, 'Year': 0, 'Runtime': 1, 'imdbRating': 0, 'BoxOffice': 0, 'Genre': 0}, 'data_type_issues': {'imdbRating': 1}}

2. Cleaning...

3. Validation...
Valid Movies: 4
Invalid Movies: 4

4. Quality Checks:
Data Quality Checks:
--------------------------------------------------
✓ No duplicates: OK
✓ No empty titles: OK
✗ Valid years: Found 1 invalid years
✗ Valid ratings: Found 1 invalid ratings
✗ Valid boxoffice: Found 2 invalid BoxOffice
✗ Valid runtime: Found 1 invalid runtime
✗ Valid Nulls: Found 10.416666666666657% null rows

5. Exporting...


---

# Part 8: Challenge Problems

### Challenge 8.1: Fuzzy Duplicate Detection

Sometimes duplicates have slight variations (e.g., "The Matrix" vs "Matrix, The").

Write a function that finds potential duplicates using fuzzy string matching.

**Hint**: Use the `fuzzywuzzy` or `rapidfuzz` library.

In [ ]:
!pip install fuzzywuzzy
!pip install python-Levenshtein

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.3/153.3 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 30.3 MB/s eta 0:00:00


In [ ]:
# YOUR CODE HERE
from fuzzywuzzy import fuzz
def detection(word1, word2):
  return fuzz.ratio(word1, word2)

print(f"{detection("The Matrix", "Matrix, The")}% match")

57% match


### Challenge 8.2: Automatic Type Inference

Write a function that automatically infers the correct data type for each column by analyzing the values.

In [ ]:
# YOUR CODE HERE
def auto_type_inference(df):
  df_clean = clean_movie_data(df)
  return df_clean.convert_dtypes()

print(df_messy.info())
print(auto_type_inference(df_messy).info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Title       10 non-null     object
 1   Year        10 non-null     object
 2   Runtime     10 non-null     object
 3   imdbRating  10 non-null     object
 4   BoxOffice   10 non-null     object
 5   Genre       10 non-null     object
dtypes: object(6)
memory usage: 612.0+ bytes
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Title       8 non-null      string 
 1   Year        7 non-null      Int64  
 2   Runtime     7 non-null      Int64  
 3   imdbRating  7 non-null      Float64
 4   BoxOffice   6 non-null      Int64  
 5   Genre       8 non-null      string 
dtypes: Float64(1), Int64(3), string(2)
memory usage: 548.0 bytes
None


---

# Summary

In this lab, you learned:

1. **CLI Inspection**: Using jq and Unix tools for quick data exploration
2. **Data Profiling**: Systematic analysis of data quality issues
3. **Data Cleaning**: Converting messy strings to proper types
4. **Schema Validation**: Using Pydantic for type safety
5. **Quality Checks**: Automated assertions for data quality

## Next Week

**Week 3: Data Labeling & Annotation**
- Setting up Label Studio
- Annotation workflows
- Measuring inter-annotator agreement